In [1]:
# using ast.literal_eval()
import sys
import ast
import os
import time
import pandas as pd
import pickle
import glob
import numpy as np
import statistics
from scipy.stats import mannwhitneyu

In [2]:
with open("hPPIN_paths.dictionary", 'rb') as config_dictionary_file:
 
    # Step 3
    paths = pickle.load(config_dictionary_file)

In [3]:
# create list
file2 = "common_nodes_hPPIN_STRING_PANCAN.txt"
combined_list = []
with open(file2) as IN:
    for line in IN:
        combined_list.append(line.rstrip().split("\t")[0])

In [4]:
def compute_sp(list1, list2, paths):
    total_sp = []
    for i in range(len(list1)):
        for j in range(len(list2)):
            if(list1[i] in paths.keys()):
                shortest_paths = paths[list1[i]]
                if(list2[j] in shortest_paths.keys()):
                    total_sp.append(shortest_paths[list2[j]])
    return({"distribution": total_sp,
            "mean": statistics.mean(total_sp),
            "median": statistics.median(total_sp)})

In [5]:
# Read Cancersea Data
folder_path = "/data/timonaj/cancer_as_wound/ppi_analysis/cancersea/"

filenames = os.listdir(folder_path)

cancersea = {}

for filename in filenames:
    file_path = f"{folder_path}/{filename}"
    cancersea[filename[:-4]] = list()

    with open(file_path) as IN:
        for line in IN:
            cancersea[filename[:-4]].append(line.rstrip().split("\t")[1])

In [6]:
cancersea["COSMIC"] = list()
with open("/data/timonaj/cancer_as_wound/ppi_analysis/cosmic_hallmarks.txt") as IN:
    for line in IN:
        cancersea["COSMIC"].append(line.rstrip().split("\t")[0])

In [7]:
# read in HP sets
folder_path = "/data/timonaj/cancer_as_wound/ppi_analysis/HP_genesets/"

filenames = os.listdir(folder_path)

hp_genesets = {}

for filename in filenames:
    file_path = f"{folder_path}/{filename}"
    hp_genesets[filename[:-4]] = list()

    with open(file_path) as IN:
        for line in IN:
            hp_genesets[filename[:-4]].append(line.rstrip().split("\t")[0])

In [8]:
# read in HP matched controls set
folder_path = "/data/timonaj/cancer_as_wound/ppi_analysis/HP_genesets_matched_controls/"

filenames = glob.glob("/data/timonaj/cancer_as_wound/ppi_analysis/HP_genesets_matched_controls/hPPIN*")
hp_genesets_matched = {}

for filename in filenames:
    name = filename.split("hPPIN_")[1][:-4]
    hp_genesets_matched[name] = list()

    with open(filename) as IN:
        for line in IN:
            hp_genesets_matched[name].append(line.rstrip().split("\t")[0])

In [9]:
a_mean = np.zeros(shape=(len(hp_genesets), len(cancersea)))
a_median = np.zeros(shape=(len(hp_genesets), len(cancersea)))
a_dist = pd.DataFrame(index = range(len(cancersea)),
                      columns = range(len(hp_genesets)))

In [10]:
i=0
for key1,value1 in hp_genesets.items():
    j=0
    for key2,value2 in cancersea.items():
        current_info = compute_sp(value1, value2, paths)
        a_mean[i][j] = current_info["mean"]
        a_median[i][j] = current_info["median"]
        a_dist[i][j] = current_info["distribution"]
        j+=1
    i+=1

In [11]:
pd.DataFrame(a_median)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
1,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
4,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
5,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
6,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
7,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
8,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [12]:
pd.DataFrame(a_mean)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,2.819539,2.864208,2.892614,2.811578,2.737534,2.743152,2.950091,2.820000,2.821481,2.957830,2.959289,2.869451,2.801728,2.881238,2.825073
1,2.954240,2.968972,3.041715,2.955626,2.899500,2.852086,3.102891,2.954380,2.992496,3.106340,3.077789,2.988591,2.898523,3.041039,2.950114
2,2.913497,2.940732,2.967998,2.911037,2.833542,2.825419,3.061532,2.918678,2.941205,3.035640,3.046925,2.967078,2.878020,2.974478,2.916098
3,2.747641,2.870351,2.803568,2.756787,2.636234,2.723623,2.882535,2.783151,2.742894,2.876210,2.903709,2.818946,2.798114,2.766481,2.772855
4,2.722232,2.794367,2.796803,2.719504,2.643310,2.657214,2.886265,2.738858,2.751305,2.869728,2.867646,2.783288,2.725530,2.773527,2.743050
5,2.896626,2.924003,2.968872,2.894249,2.820917,2.808739,3.037768,2.897491,2.919253,3.032471,3.031020,2.945032,2.862812,2.964950,2.898832
6,2.861625,2.901236,2.942167,2.862926,2.791213,2.772225,3.006074,2.859977,2.885651,3.005762,2.997202,2.907008,2.828287,2.934758,2.863167
7,2.714846,2.725670,2.812233,2.708130,2.656730,2.590196,2.881693,2.694920,2.752457,2.876824,2.858237,2.777933,2.650960,2.804289,2.711501
8,3.006166,3.024599,3.095687,3.006777,2.948687,2.898435,3.152673,2.995503,3.043984,3.153494,3.125839,3.031224,2.945445,3.092959,2.996696


In [125]:
pd.DataFrame(a_mean, columns = cancersea.keys(), index = hp_genesets.keys()).to_csv("hPPIN_meanSP_HP_Phenotypes.csv")

In [126]:
a_matched_mean = np.zeros(shape=(len(hp_genesets_matched), len(cancersea)))
a_matched_median = np.zeros(shape=(len(hp_genesets_matched), len(cancersea)))
a_matched_dist = pd.DataFrame(index = range(len(cancersea)),
                              columns = range(len(hp_genesets_matched)))

In [127]:
i=0
for key1,value1 in hp_genesets_matched.items():
    j=0
    for key2,value2 in cancersea.items():
        current_info = compute_sp(value1, value2, paths)
        a_matched_mean[i][j] = current_info["mean"]
        a_matched_median[i][j] = current_info["median"]
        a_matched_dist[i][j] = current_info["distribution"]
        j+=1
    i+=1

In [128]:
pd.DataFrame(a_matched_median)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
1,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
4,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
5,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
6,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
7,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
8,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [129]:
pd.DataFrame(a_matched_mean)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,2.979792,2.978069,3.078124,2.977233,2.931626,2.863447,3.130305,2.974983,3.031669,3.129733,3.105763,3.028115,2.902401,3.067041,2.969573
1,2.984516,2.983632,3.088898,2.982721,2.939574,2.873419,3.138165,2.980920,3.036986,3.141154,3.112983,3.035191,2.910656,3.074201,2.975673
2,3.129978,3.123841,3.224111,3.131588,3.084180,3.013937,3.270970,3.122074,3.175246,3.274613,3.253671,3.176803,3.051249,3.226543,3.115585
3,3.016479,3.014604,3.112564,3.014580,2.967986,2.907136,3.158140,3.018628,3.061887,3.164926,3.139757,3.061138,2.942966,3.104367,3.006801
4,3.264468,3.249538,3.346104,3.272468,3.215679,3.145989,3.418735,3.251066,3.309061,3.403987,3.390438,3.304550,3.175385,3.355347,3.246623
5,3.011738,3.005215,3.108037,3.006921,2.963218,2.893361,3.160836,3.005257,3.057220,3.160912,3.133656,3.052055,2.930257,3.099100,2.998375
6,2.988227,2.992730,3.082024,2.985277,2.937163,2.884553,3.131598,2.993410,3.026956,3.139309,3.115272,3.040148,2.924913,3.073967,2.980723
7,2.962230,2.960637,3.058231,2.958472,2.911202,2.850295,3.112608,2.958240,3.005323,3.113509,3.090253,3.012701,2.884850,3.045774,2.951416
8,2.980112,2.979975,3.074339,2.977643,2.928057,2.865276,3.123311,2.976067,3.020196,3.124955,3.103858,3.024446,2.905668,3.061743,2.967716


In [130]:
pd.DataFrame(a_matched_mean, columns = cancersea.keys(), index = hp_genesets_matched.keys()).to_csv("hPPIN_meanSP_HPmatched_Phenotypes.csv")

In [131]:
mannwhitneyu_p = np.zeros(shape=(len(hp_genesets), len(cancersea)))
test_dist = np.zeros(shape=(len(hp_genesets), len(cancersea)))
test_matched_dist = np.zeros(shape=(len(hp_genesets), len(cancersea)))

In [132]:
for i in range(len(hp_genesets)):
    for j in range(len(cancersea)):
        actual_dist = a_dist[i][j]
        control_dist = a_matched_dist[i][j]
        res = mannwhitneyu(actual_dist, control_dist, alternative="less")
        mannwhitneyu_p[i][j] = res.pvalue
        test_dist[i][j] = statistics.mean(actual_dist)
        test_matched_dist[i][j] = statistics.mean(control_dist)

In [133]:
cancersea.keys()

dict_keys(['Differentiation', 'Cell_Cycle', 'EMT', 'Proliferation', 'Metastasis', 'DNA_damage', 'Quiescence', 'Apoptosis', 'Inflammation', 'Invasion', 'Stemness', 'Hypoxia', 'DNA_repair', 'Angiogenesis', 'COSMIC'])

In [134]:
pvals = pd.DataFrame(mannwhitneyu_p, columns = cancersea.keys(), index = hp_genesets.keys())

In [138]:
pvals

,Differentiation,Cell_Cycle,EMT,Proliferation,Metastasis,DNA_damage,Quiescence,Apoptosis,Inflammation,Invasion,Stemness,Hypoxia,DNA_repair,Angiogenesis,COSMIC
downregulated.wound,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.055498e-256,0.000000e+00,0.000000e+00
upregulated.stress,2.757233e-08,9.991989e-01,1.501028e-14,6.304324e-03,1.445162e-21,1.630598e-01,1.189138e-06,8.468859e-02,5.069425e-19,5.387057e-06,5.749685e-12,7.974398e-17,9.811686e-01,9.149892e-05,1.576161e-10
downregulated.regen,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
msigdb.wound,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
msigdb.regen,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
upregulated.regen,0.000000e+00,6.265584e-134,7.174734e-284,1.601777e-174,0.000000e+00,2.796269e-123,7.589396e-153,4.585779e-133,0.000000e+00,3.421467e-231,5.567860e-232,6.096063e-154,1.455790e-95,4.634174e-179,0.000000e+00
upregulated.wound,0.000000e+00,5.784651e-168,2.341791e-259,2.042006e-189,0.000000e+00,1.685010e-201,2.619543e-137,4.153522e-184,0.000000e+00,3.688020e-225,2.692857e-284,1.063021e-218,4.334241e-175,2.611125e-170,0.000000e+00
msigdb.stress,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
downregulated.stress,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.995312e-01,1.000000e+00,1.000000e+00,1.000000e+00


In [135]:
pvals.to_csv("hPPIN_matched_pvals.csv")

In [136]:
pd.DataFrame(test_dist)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,2.819539,2.864208,2.892614,2.811578,2.737534,2.743152,2.950091,2.820000,2.821481,2.957830,2.959289,2.869451,2.801728,2.881238,2.825073
1,2.954240,2.968972,3.041715,2.955626,2.899500,2.852086,3.102891,2.954380,2.992496,3.106340,3.077789,2.988591,2.898523,3.041039,2.950114
2,2.913497,2.940732,2.967998,2.911037,2.833542,2.825419,3.061532,2.918678,2.941205,3.035640,3.046925,2.967078,2.878020,2.974478,2.916098
3,2.747641,2.870351,2.803568,2.756787,2.636234,2.723623,2.882535,2.783151,2.742894,2.876210,2.903709,2.818946,2.798114,2.766481,2.772855
4,2.722232,2.794367,2.796803,2.719504,2.643310,2.657214,2.886265,2.738858,2.751305,2.869728,2.867646,2.783288,2.725530,2.773527,2.743050
5,2.896626,2.924003,2.968872,2.894249,2.820917,2.808739,3.037768,2.897491,2.919253,3.032471,3.031020,2.945032,2.862812,2.964950,2.898832
6,2.861625,2.901236,2.942167,2.862926,2.791213,2.772225,3.006074,2.859977,2.885651,3.005762,2.997202,2.907008,2.828287,2.934758,2.863167
7,2.714846,2.725670,2.812233,2.708130,2.656730,2.590196,2.881693,2.694920,2.752457,2.876824,2.858237,2.777933,2.650960,2.804289,2.711501
8,3.006166,3.024599,3.095687,3.006777,2.948687,2.898435,3.152673,2.995503,3.043984,3.153494,3.125839,3.031224,2.945445,3.092959,2.996696


In [137]:
pd.DataFrame(test_matched_dist)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,2.979792,2.978069,3.078124,2.977233,2.931626,2.863447,3.130305,2.974983,3.031669,3.129733,3.105763,3.028115,2.902401,3.067041,2.969573
1,2.984516,2.983632,3.088898,2.982721,2.939574,2.873419,3.138165,2.980920,3.036986,3.141154,3.112983,3.035191,2.910656,3.074201,2.975673
2,3.129978,3.123841,3.224111,3.131588,3.084180,3.013937,3.270970,3.122074,3.175246,3.274613,3.253671,3.176803,3.051249,3.226543,3.115585
3,3.016479,3.014604,3.112564,3.014580,2.967986,2.907136,3.158140,3.018628,3.061887,3.164926,3.139757,3.061138,2.942966,3.104367,3.006801
4,3.264468,3.249538,3.346104,3.272468,3.215679,3.145989,3.418735,3.251066,3.309061,3.403987,3.390438,3.304550,3.175385,3.355347,3.246623
5,3.011738,3.005215,3.108037,3.006921,2.963218,2.893361,3.160836,3.005257,3.057220,3.160912,3.133656,3.052055,2.930257,3.099100,2.998375
6,2.988227,2.992730,3.082024,2.985277,2.937163,2.884553,3.131598,2.993410,3.026956,3.139309,3.115272,3.040148,2.924913,3.073967,2.980723
7,2.962230,2.960637,3.058231,2.958472,2.911202,2.850295,3.112608,2.958240,3.005323,3.113509,3.090253,3.012701,2.884850,3.045774,2.951416
8,2.980112,2.979975,3.074339,2.977643,2.928057,2.865276,3.123311,2.976067,3.020196,3.124955,3.103858,3.024446,2.905668,3.061743,2.967716
